In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import transformers as tf
from sklearn.linear_model import LogisticRegression

In [2]:
DATA_PATH = '../data/snapshots/train.csv'
df = pd.read_csv(DATA_PATH)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.shape

(891, 12)

In [4]:
X = df.drop('Survived', axis=1)
y = df['Survived']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
cat_columns = ['Name', 'Sex', 'Ticket', 'Embarked']

In [7]:
num_columns=['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [8]:
drop_list = [
    'PassengerId',
    'Cabin',
    'Name',
    'Ticket'
]

In [9]:
cat_imputer = tf.FeatureTransformer(
    [
        ('cat_imputer',tf.Imputer(method='most_frequent'), cat_columns),
    ]
)

cat_encoder = tf.FeatureTransformer(
    [
        ('cat_encoder',tf.CategoricalEncoder(method='nominal', drop_first='first'), cat_columns),
    ]
)

In [10]:
num_imputer = tf.FeatureTransformer(
    [
        ('num_imputer',tf.Imputer(method='median'), num_columns),
    ]
)

In [11]:
pipe = Pipeline(steps=[
    ('remover', tf.Remover(drop_list)),
    ('num_imputer', num_imputer),
    ('cat_imputer', cat_imputer),
    ('cat_encoder', cat_encoder)
])

In [12]:
pipe = pipe.fit(X_train,y)

2021-07-18 22:30:40,525 - The following features: ['Ticket', 'Name'] were removed because they were no longer present in the data
2021-07-18 22:30:40,541 - The following features: ['Ticket', 'Name'] were removed because they were no longer present in the data


In [13]:
X_train_pre = pipe.transform(X_train)
X_test_pre = pipe.transform(X_test)

In [14]:
model = LogisticRegression(max_iter=1000).fit(X_train_pre,y_train)

# Save Pipeline

In [15]:
pipeline_filename = '../data/artifacts/preprocessing_pipeline.sav'

In [16]:
with open (pipeline_filename, 'wb') as file:
    pickle.dump(pipe, file)

# Save Model

In [17]:
model_filename = '../data/artifacts/model.sav'

In [18]:
with open (model_filename, 'wb') as file:
    pickle.dump(model, file)

# Test pipeline

In [19]:
with open(pipeline_filename, 'rb') as file:
    pkl_pipe = pickle.load(file)

In [20]:
pkl_pipe.transform(X_test)

,Sex_male,Embarked_Q,Embarked_S,Pclass,Age,SibSp,Parch,Fare
709,1,0,0,3,28.0,1,1,15.2458
439,1,0,1,2,31.0,0,0,10.5
840,1,0,1,3,20.0,0,0,7.925
720,0,0,1,2,6.0,0,1,33.0
39,0,0,0,3,14.0,1,0,11.2417
...,...,...,...,...,...,...,...,...
715,1,0,1,3,19.0,0,0,7.65
525,1,1,0,3,40.5,0,0,7.75
381,0,0,0,3,1.0,0,2,15.7417
140,0,0,0,3,28.0,0,2,15.2458


# Test model

In [21]:
with open(model_filename, 'rb') as file:
    pkl_model = pickle.load(file)

In [22]:
pkl_model.predict(X_test_pre)

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0])